In [25]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
%cd /content/drive/MyDrive/justnlp/summarization

/content/drive/MyDrive/justnlp/summarization


# Train Data

In [27]:
import json
import pandas as pd
def load_jsonl(path):
    records = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            records.append(json.loads(line))
    return pd.DataFrame(records)

train_path='./dataset/train/train_judg.jsonl'
judg=load_jsonl(train_path)

In [ ]:
judg.head()

,ID,Judgment
0,id_10,Case :- WRIT - C No. - 11383 of 2023\nPetition...
1,id_1000,Versus\nAppearance:\nand\nDate : 22/09/2023\n1...
2,id_1001,Non-Reportable\nCriminal Appeal No._________ o...
3,id_1002,---- Appellant\nVersus\nUmesh Sharma S/o Late...
4,id_1003,1Whether Reporters of Local Papers may be allo...


In [ ]:
judg.shape

(1200, 2)

In [28]:
import nltk
nltk.download("punkt_tab")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
judg["word_count"] = judg["Judgment"].apply(lambda x: len(nltk.word_tokenize(x)))
judg["sentence_count"] = judg["Judgment"].apply(lambda x: len(nltk.sent_tokenize(x)))
print(judg[["ID", "word_count", "sentence_count"]])

           ID  word_count  sentence_count
0       id_10        3949             258
1     id_1000        1938              51
2     id_1001        1574              56
3     id_1002        2386              79
4     id_1003        2008              60
...       ...         ...             ...
1195   id_995        8844             309
1196   id_996        9094             346
1197   id_997        1135              33
1198   id_998        2194              80
1199   id_999        2854             100

[1200 rows x 3 columns]


In [ ]:
judg['word_count'].min()

169

In [ ]:
judg['word_count'].max()

149745

In [ ]:
judg['word_count'].mean()

np.float64(8368.1275)

In [ ]:
judg['Judgment'].isnull().sum()

np.int64(0)

In [ ]:
print(judg['sentence_count'].min())
print(judg['sentence_count'].max())
print(judg['sentence_count'].mean())

1
5690
284.4691666666667


In [ ]:
import re
def clean_text(df, column):
  df[column]=df[column].apply(lambda x: x.lower())
  df[column] = df[column].apply(lambda x: re.sub(r'http|https:\S+|www\.\S+', '', x))
  df[column] = df[column].apply(lambda x:re.sub(r'[^\x00-\x7F]+', ' ', x))
  df[column] = df[column].apply(lambda x: re.sub(r'[^a-z0-9.?\s]', ' ', x))
  df[column] = df[column].apply(lambda x: re.sub(r'-\s*\n\s*', '', x))
  df[column] = df[column].apply(lambda x: re.sub(r'\n{2,}', '\n\n', x))
  df[column]=df[column].apply(lambda x: re.sub(r'([^\n])\n([^\n])', r'\1 \2',x))
  df[column] = df[column].apply(lambda x:  re.sub(r'[^\x09\x0A\x0D\x20-\x7E]', ' ', x))
  df[column] = df[column].apply(lambda x: re.sub(r"\s+", " ", x))
  print("after cleaning")
  df["word_count"] = df[column].apply(lambda x: len(nltk.word_tokenize(x)))
  df["sentence_count"] = df[column].apply(lambda x: len(nltk.sent_tokenize(x)))
  print(df[["ID", "word_count","sentence_count"]])
  print(df['word_count'].min())
  print(df['word_count'].max())
  print(df['word_count'].mean())
  return df

In [ ]:
judg=clean_text(judg,'Judgment')

after cleaning
           ID  word_count  sentence_count
0       id_10        3496             291
1     id_1000        1856              52
2     id_1001        1465              56
3     id_1002        2297              77
4     id_1003        1899              59
...       ...         ...             ...
1195   id_995        7797             256
1196   id_996        8534             268
1197   id_997        1063              27
1198   id_998        2049              57
1199   id_999        2680              82

[1200 rows x 3 columns]
161
140293
7728.201666666667


In [ ]:
from collections import Counter
def extract_sentences(docs):
    all_sents = []
    one_sents=[]
    for doc in docs:
        sentences = nltk.sent_tokenize(doc)

        one_sent=[s for s in sentences if len(s.split()) == 1]
        #print(one_sent)
        one_sents.extend(one_sent)
        all_sents.extend(sentences)
    return all_sents,one_sents

# Collect sentences from all judgments
all_sents,one_sents= extract_sentences(judg["Judgment"])

# Count most common sentences
sent_counter = Counter(all_sents)

# Show top 20
print(sent_counter.most_common(100))


[('no.', 867), ('.', 558), ('ii.', 236), ('explanation.', 183), ('iii.', 182), ('iv.', 134), ('nos.', 127), ('154 of 20 20 vinod dua vs. union of india & ors.', 117), ('case no.', 113), ('addl.', 110), ('page no.', 96), ('the authenticity of the order can be re-verified from delhi high court order portal by scanning the qr code shown above.', 93), ('w.p.', 86), ('(c) no.', 83), ('the ld.', 81), ('07/2019 cnr no.', 81), ('ld.', 76), ('vi.', 72), ('i.a.', 67), ('sh.', 65), ('& ors.', 65), ('(cr.)', 56), ('vii.', 56), ('no objection to the premature release of the prisoner.', 56), ('[emphasis is mine.]', 56), ('act.', 48), ('31/2021 page no.', 47), ('competition appeal (at) no.', 43), ('viii.', 41), ('misc.', 41), ('no costs.', 39), ('kunal kamra v union of india & connected matters per gs patel j oswpl -9792 -2023++j-f- gspatelj.', 39), ('ix.', 38), ('explanation .', 37), ('thru.', 36), ('dg (int.)', 35), ('respondent no.', 35), ('xi.', 34), ('illustrations.', 34), ('o.p.', 33), ('wp no.

In [ ]:
one_sents

['thru.',
 'addl.',
 'chief/prin.',
 'secy.',
 'lko.',
 'thru.',
 'addl.',
 'secy.',
 'thru.',
 'addl.',
 'chief/prin.',
 'secy.',
 'lko.',
 'thru.',
 'addl.',
 'chief/prin.',
 'secy.',
 'lko.',
 'thru.',
 'addl.',
 'chief/prin.',
 'secy.',
 'lko.',
 'thru.',
 'addl.',
 'chief/prin.',
 'secy.',
 'u.p.',
 'thru.',
 'addl.',
 'chief/prin.',
 'secy.',
 'lko.',
 'thru.',
 'addl.',
 'chief/prin.',
 'secy.',
 'u.p.',
 'thru.',
 'addl.',
 'secy.',
 'thru.',
 'addl.',
 'chief/prin.',
 'secy.',
 'lko.',
 'thru.',
 'chief/prin.',
 'secy.',
 'lko.',
 'thru.',
 'addl.',
 'secy.',
 'thru.',
 'addl.',
 'chief/prin.',
 'secy.',
 'thru.',
 'addl.',
 'chief/prin.',
 'secy.',
 'thru.',
 'addl.',
 'thru.',
 'addl.',
 'chief/prin.',
 'secy.',
 'thru.',
 'addl.',
 'thru.',
 'addl.',
 'thru.',
 'addl.',
 'secy.',
 'thru.',
 'addl.',
 'u.p.',
 'lko.',
 'thru.',
 'addl.',
 'secy.',
 'thru.',
 'addl.',
 'chief/prin.secy.',
 'thru.',
 'addl.',
 'thru.',
 'prin./addl.',
 'lko.',
 'thru.',
 'addl.',
 '/prin.',
 '

In [ ]:
sent_counter=sent_counter.most_common(100)
sent_boiler=[s for s in sent_counter if len(s[0].split())<6]

In [ ]:
all_sent_boiler_items = [s for s, _ in sent_boiler]
print(all_sent_boiler_items)

['no.', '.', 'ii.', 'explanation.', 'iii.', 'iv.', 'nos.', 'case no.', 'addl.', 'page no.', 'w.p.', '(c) no.', 'the ld.', '07/2019 cnr no.', 'ld.', 'vi.', 'i.a.', 'sh.', '& ors.', '(cr.)', 'vii.', '[emphasis is mine.]', 'act.', '31/2021 page no.', 'competition appeal (at) no.', 'viii.', 'misc.', 'no costs.', 'ix.', 'explanation .', 'thru.', 'dg (int.)', 'respondent no.', 'xi.', 'illustrations.', 'o.p.', 'wp no.', 'secy.', 'mtrs.', 'smt.', 'writ petition (criminal) no.', 'm/s.', 'of gujarat.', '(supra).', 'illustration .', '02/2021 bhagwan shrikrishna virajman etc.', 'vs. u.p.', 'sunni central waqf board etc.', '(kat).', 'of india.', 'p .', '25-10-2011).', 'l.j.', 'xii.', ". '", 'illustration.', 'mr. vice-president: amendment no .', 'of gujarat, from the addl.', '12228/2021 cnr no.', 'dlse-02012234-2021 page no.', 'xiii.', 'high court of h.p.', 'fir no.', 'p.c.', 'the respondent no.', 'adv.', '2.', 'etc.', 'saibaba.', 'defendant no.', 'of !', 'ka.', '3.', 'leave granted.', 'appeal no.',

In [ ]:
all_sent_boiler_items.extend(one_sents)


In [ ]:
all_sent_boiler_items=list(set(all_sent_boiler_items))

In [ ]:
all_sent_boiler_items

['further.',
 '7/1.',
 '850/2008.',
 'xiii.',
 'furnish~d.',
 'r.s.',
 'ref.',
 '1692/201194.',
 '17.)',
 '127.mr.',
 'w.p(crl).',
 '39.',
 '1/20231.',
 'through:sh.',
 '4.5mr.',
 '182.mr.',
 '54.dr.',
 'allowed.',
 '12.ld.',
 '06.1996.',
 '124a.',
 'ltd.8.',
 '62.',
 'sub.',
 'cr.p.c.',
 'distt.',
 '(iv).',
 '20(ii).',
 'washim.',
 '3.mr.',
 '32.mr.',
 '128.mr.',
 'opp.',
 'lvii.',
 'd.d.',
 'crl.rev.pet.',
 '41.',
 'nangia102.',
 'ff.',
 '3]shri.',
 'and.',
 'rona.',
 '10.mr.',
 'xix.',
 '19.mr.',
 'a(1).',
 'mat.',
 'r.p.',
 '23(ix).',
 'bangalore',
 '29.09.2023',
 'asj.',
 'departmental.',
 '4,50,000/-.',
 '44.',
 'ans6.',
 '16.09.2022',
 'meshram.',
 '869.',
 '(vol.',
 '25.',
 'u.d.',
 'wp(crl.)',
 '(xiv).',
 '279.mr.',
 '110.mr.',
 '[26a.',
 'lll.',
 '4.ms.',
 'ka-2.',
 'v.v.',
 '20."',
 '11/2020.',
 '4.3mr.',
 '24/2013.',
 '(b.h.u.)',
 '493/2022].',
 'xvii.',
 '(xix).',
 'n.n.',
 '111.',
 '(4)md.',
 '16498-a.',
 '118.',
 '(2.)',
 'appl.',
 '1232(e).',
 'xlvi.',
 '2016).',
 'ms.h

In [ ]:
from collections import Counter
import re

def tokenize_words(text):
    text = text.lower()
    words = re.findall(r'\b[a-z\.]+\b', text)
    return words

all_tokens = []
for doc in judg["Judgment"]:
    all_tokens.extend(tokenize_words(str(doc)))

word_freq = Counter(all_tokens)
print(word_freq.most_common(50))


[('the', 770515), ('of', 443642), ('to', 242121), ('and', 201556), ('in', 189362), ('a', 133253), ('that', 123287), ('is', 123255), ('be', 84236), ('by', 84024), ('for', 77443), ('on', 74264), ('as', 71338), ('or', 65660), ('it', 60262), ('not', 58109), ('was', 55853), ('court', 54963), ('.', 51554), ('this', 46305), ('with', 46083), ('has', 43005), ('which', 42219), ('under', 37369), ('section', 34477), ('any', 33175), ('an', 33145), ('are', 32578), ('act', 32565), ('no', 29299), ('from', 28708), ('state', 27722), ('have', 26798), ('at', 26387), ('case', 25947), ('been', 25433), ('such', 25315), ('s', 24325), ('he', 23983), ('also', 21147), ('would', 19770), ('order', 19602), ('his', 18663), ('may', 17839), ('other', 17481), ('law', 17263), ('shall', 17246), ('there', 17232), ('had', 17020), ('if', 15905)]


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000, stop_words="english")
X = vectorizer.fit_transform(judg["Judgment"].astype(str))

avg_tfidf = np.asarray(X.mean(axis=0)).ravel()
terms = vectorizer.get_feature_names_out()

# Sort by lowest average tf-idf → most likely boilerplate
low_tfidf_terms = [t for t, score in sorted(zip(terms, avg_tfidf), key=lambda x: x[1])[:50]]
print("\n🔹 Candidate boilerplate terms (low TF-IDF):")
print(low_tfidf_terms)


NameError: name 'np' is not defined

In [ ]:
boilerplate_dict = {
    "sentences": all_sent_boiler_items,
    "terms": ['fron', 'ebix', 'hin', 'rpa', 'fma', 'spv', 'axykno', 'thot', 'apte','ond', 'jv', 'deccan']
}

import json
with open("boilerplate_candidates.json", "w", encoding="utf-8") as f:
    json.dump(boilerplate_dict, f, indent=2, ensure_ascii=False)

print("\n✅ Boilerplate candidates saved to boilerplate_candidates.json")


✅ Boilerplate candidates saved to boilerplate_candidates.json


In [29]:
train_ref_path='./dataset/train/train_ref_summ.jsonl'
ref=load_jsonl(train_ref_path)

In [ ]:
ref.head()

,ID,Summary
0,id_10,The Allahabad High Court on Thursday stayed th...
1,id_1000,A convict in Gujarat who had secured bail in 2...
2,id_1001,A police officer failing in their fundamental ...
3,id_1002,The Chhattisgarh High Court recently observed ...
4,id_1003,The Gujarat High Court recently quashed a Firs...


In [ ]:
ref.shape

(1200, 2)

In [ ]:
ref["word_count"] = ref["Summary"].apply(lambda x: len(nltk.word_tokenize(x)))
ref["sentence_count"] = ref["Summary"].apply(lambda x: len(nltk.sent_tokenize(x)))

print(ref[["ID", "word_count", "sentence_count"]])

           ID  word_count  sentence_count
0       id_10         759              22
1     id_1000         595              17
2     id_1001         557              17
3     id_1002         444              17
4     id_1003         486              16
...       ...         ...             ...
1195   id_995         775              26
1196   id_996         790              20
1197   id_997         580              18
1198   id_998         463              14
1199   id_999         679              23

[1200 rows x 3 columns]


In [ ]:
print(ref['word_count'].min())
print(ref['word_count'].max())
print(ref['word_count'].mean())

29
2412
609.9591666666666


In [ ]:
print(ref['sentence_count'].min())
print(ref['sentence_count'].max())
print(ref['sentence_count'].mean())

1
73
19.948333333333334


In [ ]:
ref['Summary'].isnull().sum()

np.int64(0)

In [ ]:
nltk.sent_tokenize(ref['Summary'].iloc[10])

['The Supreme Court on Monday granted bail to Sadanand Kadam, an aide of Shiv Sena (UBT) MLC Anil Parab in the Dapoli resort money laundering case.',
 "[Sadanand Gangaram Kadam v Directorate of Enforcement and Anr]\nA bench of Justices Abhay S Oka and Ujjal Bhuyan noted Additional Solicitor General (ASG) Suryaprakash V Raju's fair stand of leaving it to the Court to decide the prayer for bail.",
 'Even otherwise, the Court found that Kadam was entitled to bail in accordance with Section 45(1)(ii) of the Prevention of Money Laundering Act (PMLA) on appropriate terms and conditions, till the disposal of the complaint case filed by the Enforcement Directorate.',
 "In view of the ASG's stand, the Court did not record reasons in its order and directed that Kadam be produced before the Special Court within a week for being released on bail on appropriate terms and conditions.",
 'In March 2022, the income tax (IT) department had raided premises in several districts of Maharashtra linked to a

In [ ]:
judg

,ID,Judgment,word_count,sentence_count
0,id_10,case writ c no. 11383 of 2023 petitioner syed ...,3496,291
1,id_1000,versus appearance and date 22 09 2023 1.the pr...,1856,52
2,id_1001,non reportable criminal appeal no. of 2024 spe...,1465,56
3,id_1002,appellant versus umesh sharma s o late omprak...,2297,77
4,id_1003,1whether reporters of local papers may be allo...,1899,59
...,...,...,...,...
1195,id_995,arising out of special leave petition crimina...,7797,256
1196,id_996,reserved on 15.03.2023 pronounced on 16.05.20...,8534,268
1197,id_997,reportable writ petition c no 961 of 2021 neil...,1063,27
1198,id_998,m s. biovet private limited .applicants in the...,2049,57


In [30]:
ref=clean_text(ref,'Summary')

after cleaning
           ID  word_count  sentence_count
0       id_10         727              22
1     id_1000         571              16
2     id_1001         528              17
3     id_1002         423              17
4     id_1003         469              16
...       ...         ...             ...
1195   id_995         707              26
1196   id_996         744              22
1197   id_997         562              18
1198   id_998         452              13
1199   id_999         638              22

[1200 rows x 3 columns]
27
2181
571.2291666666666


In [ ]:
ref

,ID,Summary,word_count,sentence_count
0,id_10,the allahabad high court on thursday stayed th...,727,22
1,id_1000,a convict in gujarat who had secured bail in 2...,571,16
2,id_1001,a police officer failing in their fundamental ...,528,17
3,id_1002,the chhattisgarh high court recently observed ...,423,17
4,id_1003,the gujarat high court recently quashed a firs...,469,16
...,...,...,...,...
1195,id_995,facts sourced from a statement made by accused...,707,26
1196,id_996,the delhi high court recently directed mediati...,744,22
1197,id_997,the supreme court on friday passed an interim ...,562,18
1198,id_998,the bombay high court on thursday permitted bi...,452,13


In [ ]:
merged_df = judg.merge(ref, on="ID", how="inner")

In [ ]:
merged_df = merged_df[['ID','Judgment','Summary']]
merged_df

,ID,Judgment,Summary
0,id_10,case writ c no. 11383 of 2023 petitioner syed ...,the allahabad high court on thursday stayed th...
1,id_1000,versus appearance and date 22 09 2023 1.the pr...,a convict in gujarat who had secured bail in 2...
2,id_1001,non reportable criminal appeal no. of 2024 spe...,a police officer failing in their fundamental ...
3,id_1002,appellant versus umesh sharma s o late omprak...,the chhattisgarh high court recently observed ...
4,id_1003,1whether reporters of local papers may be allo...,the gujarat high court recently quashed a firs...
...,...,...,...
1195,id_995,arising out of special leave petition crimina...,facts sourced from a statement made by accused...
1196,id_996,reserved on 15.03.2023 pronounced on 16.05.20...,the delhi high court recently directed mediati...
1197,id_997,reportable writ petition c no 961 of 2021 neil...,the supreme court on friday passed an interim ...
1198,id_998,m s. biovet private limited .applicants in the...,the bombay high court on thursday permitted bi...


In [ ]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(merged_df, test_size=0.2, random_state=42, shuffle=True)

In [ ]:
import json

def save_jsonl(df, path):
    with open(path, "w", encoding="utf-8") as f:
        for _, row in df.iterrows():
            f.write(json.dumps(row.to_dict(), ensure_ascii=False) + "\n")

# Save to JSONL
# save_jsonl(train_df, "./dataset/train/train_split.jsonl")
# save_jsonl(val_df, "./dataset/train/val_split.jsonl")

# print("Train/Test JSONL files saved!")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
print(train_df.shape)
print(val_df.shape)

(960, 3)
(240, 3)


In [ ]:
train_df

,ID,Judgment,Summary
331,id_1498,and w.m.p. md no.21615 of 2022 s.lawrence vima...,designation of a lawyer as a senior advocate i...
409,id_16,this file relates to reconsideration of the pr...,the union law minister told the supreme court ...
76,id_1106,1pa. rg misc. 2021 date 24.06.202l considering...,the rajasthan high court will start regular he...
868,id_539,in the high court of madhya pradeshat jabalpur...,the madhya pradesh high court recently quashed...
138,id_1205,crl. a. no.1200 2023 criminal appeal no.1200 o...,while society should be protected from drug re...
...,...,...,...
1044,id_797,haji abdul gani khan anr. petitioners v. union...,the supreme court on monday dismissed the peti...
1095,id_863,khalil abbas fakir .. applicant tabbasum khali...,the bombay high court recently held that a div...
1130,id_911,criminal revisional jurisdiction appellate si...,the calcutta high court recently quashed a che...
860,id_527,1 cc.no.30751 2021 dated this the 18th day of ...,an special court in bengaluru recently convict...


In [ ]:
val_df=load_jsonl('./dataset/train/val_split.jsonl')

In [ ]:
val_path='./dataset/validation/val_judg.jsonl'
val=load_jsonl(train_path)

In [ ]:
nltk.sent_tokenize(val['Judgment'].iloc[1])

['Versus\nAppearance:\nand\nDate : 22/09/2023\n1.The present case is an eye opener.',
 'The convict-\nChandanji @ Gato Chhanaji Thakor has filed the\npresent application seeking regular bail through\njail.',
 'Such application was filed by him on\n05.08.2023, which is forwarded to the Registry of\nthis Court vide communication dated 11.08.2023\nwritten by the Deputy Superintendent of Ahmedabad\nCentral Jail.',
 '2.When the matter was listed yesterday, learned\nadvocate Mr.Soni appearing for the applicant-convict\nhas invited attention of this Court to the order\ndated 29.09.2020 passed in Criminal Misc.',
 'Application\n(for suspension of sentence) No.1 of 2020 in the\ncaptioned appeal and has submitted that this Court,\nafter passing a comprehensive order, had already\nreleased the applicant on regular bail by suspending\nhis sentence under the provision of Section 389 of\nthe Code of Criminal Procedure, 1973 (for short "the\n3.The matter was ordered to be listed today and\nlearned AP

In [ ]:

val_df["word_count"] = val_df["Judgment"].apply(lambda x: len(nltk.word_tokenize(x)))
val_df["sentence_count"] = val_df["Judgment"].apply(lambda x: len(nltk.sent_tokenize(x)))

print(val_df[["ID", "word_count", "sentence_count"]])

          ID  word_count  sentence_count
0     id_972        1210              36
1     id_533        3082              78
2    id_1145         547              23
3    id_1640       12750             401
4    id_1079         803              28
..       ...         ...             ...
235  id_1568        2580             138
236   id_538        3543             123
237  id_1791       39203            1167
238   id_993        7685             234
239   id_546        6041             212

[240 rows x 3 columns]


In [ ]:
val_df['Judgment'].isnull().sum()

np.int64(0)

In [ ]:
val_df=clean_text(val_df,'Judgment')

after cleaning
          ID  word_count  sentence_count
0     id_972        1210              36
1     id_533        3082              78
2    id_1145         547              23
3    id_1640       12750             401
4    id_1079         803              28
..       ...         ...             ...
235  id_1568        2580             138
236   id_538        3543             123
237  id_1791       39203            1167
238   id_993        7685             234
239   id_546        6041             212

[240 rows x 3 columns]
166
132039
7324.75


In [ ]:

val_df

,ID,Judgment,Summary,word_count,sentence_count
0,id_972,through mr. hemant daswani ms. sauyma bajpai a...,the delhi high court has upheld a single judge...,1210,36
1,id_533,order per the hon ble the chief justice alok a...,the telangana high court recently upheld the c...,3082,78
2,id_1145,vasundhara d o praful bhojane union of india t...,the bombay high court today issued notice in a...,547,23
3,id_1640,reportable high court bar association allahaba...,a constitution bench of the supreme court on t...,12750,401
4,id_1079,applicant phoolchandra yadav and 2 others oppo...,the allahabad high court has granted bail to t...,803,28
...,...,...,...,...,...
235,id_1568,ms. kangana ranaut ...petitioner versus 1. the...,the bombay high court on friday rejected a pet...,2580,138
236,id_538,versus appearance ms urvashi k mehta 11469 for...,a rape is a rape even if it is committed by th...,3543,123
237,id_1791,reportable civil appeal no. 8129 of 2022 madhy...,the central government s refusal to renew broa...,39203,1167
238,id_993,writ petition nos.16650 of 2020 and 144 48 of ...,the madras high court on monday upheld the con...,7685,234


In [ ]:
val_df = val_df[['ID','Judgment','Summary']]
save_jsonl(val_df, "./dataset/train/val_split2.jsonl")

In [ ]:
val_df

,ID,Judgment,Summary
0,id_972,through mr. hemant daswani ms. sauyma bajpai a...,the delhi high court has upheld a single judge...
1,id_533,order per the hon ble the chief justice alok a...,the telangana high court recently upheld the c...
2,id_1145,vasundhara d o praful bhojane union of india t...,the bombay high court today issued notice in a...
3,id_1640,reportable high court bar association allahaba...,a constitution bench of the supreme court on t...
4,id_1079,applicant phoolchandra yadav and 2 others oppo...,the allahabad high court has granted bail to t...
...,...,...,...
235,id_1568,ms. kangana ranaut ...petitioner versus 1. the...,the bombay high court on friday rejected a pet...
236,id_538,versus appearance ms urvashi k mehta 11469 for...,a rape is a rape even if it is committed by th...
237,id_1791,reportable civil appeal no. 8129 of 2022 madhy...,the central government s refusal to renew broa...
238,id_993,writ petition nos.16650 of 2020 and 144 48 of ...,the madras high court on monday upheld the con...


In [ ]:
def split_to_sentences_summ(para):
    sents = tokenize.sent_tokenize(para)
    return sents

def get_chunks_data_from_docV2(doc, summ):
    doc_sents = split_to_sentences_summ(doc)
    summ_sents = split_to_sentences_summ(summ)
    return len(max(doc_sents)), len(max(summ_sents))
mlj=0
mls=0
for _, row in tqdm(df.iterrows(), total=len(df)):
         doc_id = str(row["ID"])
         judgment = row["Judgment"]
         reference = row["Summary"]
         dl, sl = get_chunks_data_from_docV2(judgment,reference)
         print(doc_id, dl)
        #  if dl>mlj:
        #    mlj=dl
        #  if sl>mls:
        #    mls=sl


  2%|▏         | 16/960 [00:00<00:06, 156.69it/s]

id_1498 154
id_16 134
id_1106 42
id_539 4
id_1205 110
id_1792 43
id_273 95
id_610 140
id_1084 181
id_1211 121
id_338 133
id_1559 432
id_1544 110
id_1206 306
id_558 36
id_791 225
id_1334 146
id_175 83
id_14 86
id_961 152
id_1709 131
id_1531 123
id_736 175
id_190 43
id_262 98
id_41 105


  5%|▍         | 44/960 [00:00<00:09, 97.82it/s] 

id_1807 22
id_1835 35
id_600 277
id_1681 106
id_1852 156
id_40 369
id_216 158
id_499 15
id_1430 146
id_105 128
id_1039 140
id_1358 165
id_1629 147
id_1202 331
id_871 231
id_532 4
id_1002 134
id_491 37


  6%|▌         | 57/960 [00:00<00:08, 105.53it/s]

...

id_1714 100
id_1562 147
id_1271 157
id_398 582
id_132 86
id_438 37
id_840 15
id_55 131
id_1508 173
id_343 6
id_1439 130
id_754 100
id_1663 230
id_467 175
id_1406 101
id_722 123
id_1298 191
id_1236 245
id_353 239
id_1589 215
id_316 89
id_91 140
id_1822 3
id_759 53
id_763 117
id_878 56
id_1306 20
id_1044 182
id_410 259
id_1738 169
id_982 53
id_1134 350
id_579 11
id_1821 171


 98%|█████████▊| 937/960 [00:07<00:00, 133.81it/s]

id_412 76
id_1357 127
id_1013 102
id_1471 15
id_1865 68
id_1574 341
id_1244 159
id_502 225
id_1025 54
id_233 54
id_94 138
id_49 73
id_182 90
id_962 21
id_294 123
id_67 522
id_1273 242
id_693 189
id_310 20
id_1743 377
id_848 78
id_1694 84
id_520 573
id_380 7
id_1376 142
id_1026 3
id_1668 68
id_1413 18
id_668 183
id_1572 126
id_454 271
id_1515 165


100%|██████████| 960/960 [00:07<00:00, 132.61it/s]

id_404 315
id_1190 11
id_542 75
id_904 62
id_1119 113
id_1497 113
id_1682 57
id_1175 135
id_797 126
id_863 81
id_911 31
id_527 99
id_907 199


In [41]:
vdf=load_jsonl('./dataset/train/val_split2.jsonl')

In [32]:
vdf

,ID,Judgment,Summary
0,id_972,through mr. hemant daswani ms. sauyma bajpai a...,the delhi high court has upheld a single judge...
1,id_533,order per the hon ble the chief justice alok a...,the telangana high court recently upheld the c...
2,id_1145,vasundhara d o praful bhojane union of india t...,the bombay high court today issued notice in a...
3,id_1640,reportable high court bar association allahaba...,a constitution bench of the supreme court on t...
4,id_1079,applicant phoolchandra yadav and 2 others oppo...,the allahabad high court has granted bail to t...
...,...,...,...
235,id_1568,ms. kangana ranaut ...petitioner versus 1. the...,the bombay high court on friday rejected a pet...
236,id_538,versus appearance ms urvashi k mehta 11469 for...,a rape is a rape even if it is committed by th...
237,id_1791,reportable civil appeal no. 8129 of 2022 madhy...,the central government s refusal to renew broa...
238,id_993,writ petition nos.16650 of 2020 and 144 48 of ...,the madras high court on monday upheld the con...


In [34]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('sentence-transformers/bert-base-nli-mean-tokens')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [35]:
import torch

In [33]:

def constrain_summary_length(summary_text, max_words=500):
    """Trim or select most appropriate sentences to fit within max_words."""
    sentences = nltk.sent_tokenize(summary_text)
    if not sentences:
        return summary_text

    # If already short enough, return
    if len(summary_text.split()) <= max_words:
        return summary_text

    # Encode sentences and overall summary
    sentence_embeddings = sbert_model.encode(sentences, convert_to_tensor=True)
    summary_embedding = sbert_model.encode([summary_text], convert_to_tensor=True)

    # Cosine similarity
    scores = torch.nn.functional.cosine_similarity(sentence_embeddings, summary_embedding)
    ranked_idx = scores.argsort(descending=True)

    # Select best sentences until we reach ~max_words
    selected = []
    word_count = 0
    for idx in ranked_idx:
        sent = sentences[idx]
        wc = len(sent.split())
        if word_count + wc > max_words:
            break
        selected.append(sent)
        word_count += wc

    return " ".join(selected)

In [49]:

ns=constrain_summary_length(vdf['Summary'].iloc[1])

In [50]:
ns

'the telangana high court recently upheld the constitutional validity of sections 5 6 and 15 of the press and registration of books act. v venkataramanaiah v. the union of india . a division bench of chief justice alok aradhe and justice nv shravan kumar opined that the regulation of newspaper publication by these provisions was a reasonable restriction on the right to free speech and expression. while section 5 lays down the rules for publication of newspapers section 6 deals with authentication of a declaration regarding the owner printer title language and periodicity of the publication. section 15 provides for penalty if the rules are violated. the owners of law journal law animated world in 2005 challenged section 5 on the ground that it prescribes such rules only for newspapers and not books. the other two provisions were challenged for being violative of article 19 1 freedom of speech expression etc. and article 21 right to life and personal liberty of the constitution. the cour

In [52]:
vdf['Summary'].iloc[1]

'the telangana high court recently upheld the constitutional validity of sections 5 6 and 15 of the press and registration of books act. v venkataramanaiah v. the union of india . a division bench of chief justice alok aradhe and justice nv shravan kumar opined that the regulation of newspaper publication by these provisions was a reasonable restriction on the right to free speech and expression. while section 5 lays down the rules for publication of newspapers section 6 deals with authentication of a declaration regarding the owner printer title language and periodicity of the publication. section 15 provides for penalty if the rules are violated. the owners of law journal law animated world in 2005 challenged section 5 on the ground that it prescribes such rules only for newspapers and not books. the other two provisions were challenged for being violative of article 19 1 freedom of speech expression etc. and article 21 right to life and personal liberty of the constitution. the cour